# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv('output_data/CSV/cities.csv')
cities

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Barrow,90,US,1590020832,85,71.29,-156.79,23.00,12.75
1,1,Lüderitz,0,NaN,1590021217,28,-26.65,15.16,68.00,2.24
2,2,George Town,20,MY,1590021219,88,5.41,100.34,82.40,4.70
3,3,Ranong,75,TH,1590021220,94,9.97,98.63,78.80,2.24
4,4,Saskylakh,99,RU,1590020810,92,71.92,114.08,34.47,0.74
...,...,...,...,...,...,...,...,...,...,...
537,537,Washougal,90,US,1590022229,82,45.58,-122.35,57.20,6.93
538,538,Mercedes,20,AR,1590022230,97,-34.65,-59.43,64.99,0.60
539,539,Pacific Grove,1,US,1590022231,87,36.62,-121.92,60.01,1.01
540,540,Road Town,100,VG,1590022232,89,18.42,-64.62,82.00,1.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Using the Lat and Lng as locations
locations = cities[["Lat", "Lng"]].astype(float)

# Using Humidity
humidity= cities['Humidity'].astype(float)

# Removing Null Cells
cities = cities.dropna()

In [4]:
# Setting the layout for the map
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Setting the fig and adding the center point for zoom of the map
fig = gmaps.figure(layout=figure_layout, center = (42,-42), zoom_level= 3)

# Creating Heat Layer on the map based on Locations and Humidity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

# Displaying the Fig
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrowing Down the DataFrame based on some criteria:
# 280 F < Max Temp < 300 F
# Wind Speed < 1 mph
# Cloudiness = 0 %
weather_criteria_df = cities [(cities['Max Temp']<65) & (cities['Max Temp']>55) & (cities['Wind Speed']<6) & (cities['Cloudiness']==0)]

# Removing Null Cells
weather_criteria_df.dropna()

# Saving the DataFrame to a CSV
weather_criteria_df.to_csv('output_data/CSV/Weather Criteria DF.csv', header = True, index = False)

# Displaying the DataFrame
weather_criteria_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
17,17,Port Elizabeth,0,ZA,1590021245,76,-33.92,25.57,55.40,3.36
170,170,Aksu,0,CN,1590021488,37,41.12,80.26,63.55,1.19
204,204,Nsanje,0,MW,1590021528,94,-16.92,35.26,59.63,1.99
284,284,Saurimo,0,AO,1590021694,61,-9.66,20.39,62.31,5.41
323,323,Severo-Yeniseyskiy,0,RU,1590021839,45,60.37,93.04,61.90,5.44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Creating a new DataFrame based on City, Country, Lat and Lng Columns
hotel_df = weather_criteria_df[['City', 'Country', 'Lat', 'Lng']]

# Setting Lat and Lng as Locations
location = hotel_df[['Lat', 'Lng']]

# Creating empty columns as Hotel Name and Hotel Lat and Lng for showing the accurate Location of the hotel on the map
hotel_df['Hotel Name']= ""
hotel_df['Hotel_Lat']= ""
hotel_df['Hotel_Lng']= ""

# Displaying the DataFrame
hotel_df

C:\ProgramData\Anaconda30\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda30\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda30\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

,City,Country,Lat,Lng,Hotel Name,Hotel_Lat,Hotel_Lng
17,Port Elizabeth,ZA,-33.92,25.57,,,
170,Aksu,CN,41.12,80.26,,,
204,Nsanje,MW,-16.92,35.26,,,
284,Saurimo,AO,-9.66,20.39,,,
323,Severo-Yeniseyskiy,RU,60.37,93.04,,,


In [7]:
# Setting the base_url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Setting the parameters for searching the nearest Hotel in 5000 meter of the Locations
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# uUsing iterrows to iterate through pandas DataFrame
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]

    # Changing location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
   
    # Requesting the URL 
    response = requests.get(base_url, params=params).json()
   
    
    results = response['results']

    
    try:
        print(f"{results[0]['name']} is the first hotel in {row['City']}, {row['Country']}.")
        
        # Filling the Hotel Name, Hotel_Lat and Hotel_Lng columns of the DataFrame with earned Info
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Hotel_Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Hotel_Lng'] = results[0]['geometry']['location']['lng']
   

    # Printing Missing Info when there weren't any result   
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("********************")

39 On Nile Guest House is the first hotel in Port Elizabeth, ZA.
********************


C:\ProgramData\Anaconda30\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Pudong Holiday Hotel is the first hotel in Aksu, CN.
********************
Nsanje Discovery Lodge is the first hotel in Nsanje, MW.
********************
Hotel Kawissa Saurimo is the first hotel in Saurimo, AO.
********************
Aktolik is the first hotel in Severo-Yeniseyskiy, RU.
********************


In [8]:
# Displaying the DataFrame to be show that columns are filled
hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel_Lat,Hotel_Lng
17,Port Elizabeth,ZA,-33.92,25.57,39 On Nile Guest House,-33.9462,25.5787
170,Aksu,CN,41.12,80.26,Pudong Holiday Hotel,41.1418,80.2914
204,Nsanje,MW,-16.92,35.26,Nsanje Discovery Lodge,-16.9236,35.2576
284,Saurimo,AO,-9.66,20.39,Hotel Kawissa Saurimo,-9.65714,20.3921
323,Severo-Yeniseyskiy,RU,60.37,93.04,Aktolik,60.3762,93.0312


In [9]:
# Removing Null Cells
# As it couldn't find the null cells by Dropna, I had to set the non_values to Float instead of Objects and then replace them and finally use Dropna
nan_value=float("Nan")
hotel_df.replace("",nan_value, inplace = True)
hotel_df.dropna(subset = ['Hotel Name'] , inplace = True)

# Displaying the final DataFrame after removing Null Cells
hotel_df

C:\ProgramData\Anaconda30\lib\site-packages\pandas\core\frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
C:\ProgramData\Anaconda30\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Country,Lat,Lng,Hotel Name,Hotel_Lat,Hotel_Lng
17,Port Elizabeth,ZA,-33.92,25.57,39 On Nile Guest House,-33.9462,25.5787
170,Aksu,CN,41.12,80.26,Pudong Holiday Hotel,41.1418,80.2914
204,Nsanje,MW,-16.92,35.26,Nsanje Discovery Lodge,-16.9236,35.2576
284,Saurimo,AO,-9.66,20.39,Hotel Kawissa Saurimo,-9.65714,20.3921
323,Severo-Yeniseyskiy,RU,60.37,93.04,Aktolik,60.3762,93.0312


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name: </dt><dd>{Hotel Name}</dd>
<dt>City: </dt><dd>{City}</dd>
<dt>Country: </dt><dd>{Country}</dd>
</dl>
"""

# Saving the DataFrame to a CSV
hotel_df.to_csv('output_data/CSV/Hotel_DF.csv', header = True, index = False)

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Setting the Hotel_Lat and Hotel_Lng as Locations for accurate Marker Location on the map
locations = hotel_df[["Hotel_Lat", "Hotel_Lng"]]

In [11]:
# Setting the Fig
fig = gmaps.figure(layout=figure_layout, center = (42,42), zoom_level= 2)

# Creating Markers based on the Locations and with the info box of Hotel Name, Lat and Lng
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Adding marker layer ontop of heat map
fig.add_layer(heat_layer)
fig.add_layer(markers)


# Displaying Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…